In [ ]:
import os
import tarfile
from six.moves import urllib


DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

# Using Pandas to Parse CSV

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH, housing_name="housing.csv"):
    csv_path = os.path.join(housing_path, housing_name)
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

# Getting Data Description with Pandas

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

housing.hist(bins=50, figsize=(20,15))
plt.show()

# Picking a Test Set

In [ ]:
import numpy as np

# Make sure we have a reproducible code
np.random.seed(42)

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

train_set, test_set = split_train_test(housing, 0.2)
print("We now have {:d} train samples and {:d} test samples.".format(len(train_set), len(test_set)))

# Using Hashes to Maintain a Consistent Dataset

If you want to keep a consistent test and train set, then even the `np.random.seed(n)` will not work as the indices will be moved around if the dataset is updated. One way to maitain a consistent track of the test/train instances is to use their hashes to decide whether or not they will be included in a set. Can use the last `N` bytes to decide the threashold:


 - Last two bytes can go up to 255: Check would be  $ d(XX) < 255 * \lambda $
 - Last four bytes can go up to 65535: Check would be  $ d(XXXX) < 65535 * \lambda $
 
 where $ d(X...) $ converts digits to decimal.

In [ ]:
import hashlib

def test_set_check_last2(identifier, test_ratio, hash_f):
    return hash_f(np.int64(identifier)).digest()[-1] < 256 * test_ratio

def test_set_check_last4(identifier, test_ratio, hash_f):
    bites = hash_f(np.int64(identifier)).digest()[-2:]
    return int.from_bytes(bites, "little") < 65535 * test_ratio

def split_set_by_id(data, test_ratio, id_column, hash_f=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check_last2(id_, test_ratio, hash_f))
    return data.loc[~in_test_set], data.loc[in_test_set]

# Using Pandas-Generated Index as ID

In [ ]:
housing_with_id = housing.reset_index() # Adds index column
train_set, test_set = split_set_by_id(housing_with_id, 0.2, "index")

# Combining Longitude and Latitude for ID

In [ ]:
housing_with_id["id"] = housing["longitude"]*1000 + housing["latitude"]
train_set, test_set = split_set_by_id(housing_with_id, 0.2, "id")

# Check if we have none in intersection.
train_set.merge(test_set, on=["id"])

# Using Scikit-Learn to Split

Scikit-Learn offers functions to split test/train sets in the same way as we implemented above.

In [ ]:
from sklearn.model_selection import train_test_split

# This will randomly select 20% of the data as test data
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

## Creating Strata Categories 

Dividing the `median_category` income to limit the number of strata (groups), and also cap the income category to value `5` below.

In [ ]:
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

housing["income_cat"].hist()
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_index, test_index = next(split.split(housing, housing["income_cat"]))
strat_train_set = housing.loc[train_index]
strat_test_set = housing.loc[test_index]
    
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
# Remove the "income_cat" field as it's no longer needed
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Exploring the Train Data by by Copying it

In [ ]:
housing = strat_train_set.copy()

# Clearly plotting the geographical location along
# with other information such as area population
housing.plot(
    kind="scatter",
    x="longitude",
    y="latitude",
    alpha=0.4,
    s=housing["population"]/100,
    label="population",
    c="median_house_value",
    cmap=plt.get_cmap("jet"),
    colorbar=True,
    figsize=(20,12))
plt.legend()

In [ ]:
# Calculating Correlation Matrix

correlation = housing.corr()
correlation

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(20,12))
plt.plot()

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

# Experimenting with Attribute Combination

Sometimes combining attributes is useful for feeding into machine learning algorithms. For example, combining the 
total number of rooms with the number of households will provide the average number of bedrooms per household.

In [ ]:
housing["roms_per_household"] = housing["total_rooms"] / housing["households"]
housing["beds_per_household"] = housing["total_bedrooms"] / housing["households"]
housing["population_per_household"] = housing["population"] / housing["households"]

corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# Cleaning Up and Applying Transformation to Data

Certain data transformations may be helpful on dataset. For example, some machine learning algorithms don't allow empty fields, or we may want to make the distribution of a feature more normally distributed.

We will first start by making a fresh copy of the training set.

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1).copy()
housing_labels = strat_train_set["median_house_value"].copy()

## cleaning Up Missing Fields with Vanilla Pandas

In [ ]:
# Drop only the indices (rows) with missing values in the
# subset provided
drop_missing_rows = housing.dropna(subset=["total_bedrooms"], inplace=False)

# Drop the entire feature column for a feature that has
# missing entries
drop_missing_column = housing.drop("total_bedrooms", axis=1, inplace=False)

# Fill the missing entries with the median value
# which you calculate
median = housing["total_bedrooms"].median() # Save for later use in missing tests
fill_missing_rows = housing["total_bedrooms"].fillna(median, inplace=False)

## Cleaning Up Missing Fields with Scikit's Imputer

In [ ]:
from sklearn.impute import SimpleImputer

# SimpleImputer replaced Imputer since the book has
# been released
imputer = SimpleImputer(strategy="median")

# The imputer can only work with numerical data as it
# calculated the median for each column/row
housing_num = housing.drop("ocean_proximity", axis=1, inplace=False)

# It computed the median for each attribute and saves them in
# it's statistics_ field
imputer.fit(housing_num)
imputer.statistics_

# To apply the transformation (add the median values),
# we must transform the data which creates a numpy array.
# We then put it back into a pandas DataFrame
transformed_matrix = imputer.transform(housing_num)
# Could also have done it with a single call to
# transformed_matrix = imputer.fit_transform(housing_num)
housing_tr = pd.DataFrame(transformed_matrix, columns=housing_num.columns)

## Transforming Text Attributes to Integers

In [ ]:
# Use Pandas to encode the text into integers
housing_cat = housing["ocean_proximity"]
housing_cat_encoded, category_indices = housing_cat.factorize()
print(housing_cat_encoded[:10])
print(category_indices)

## Transform Text Attributes into Sparse Binary Matrices

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(dtype=np.int16)
housing_cat_onehot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_onehot.toarray()

## Creating Custom Tranformer Classes

For a transformer, we need a class that has the follwoing methods:

* `fit()`, a a method returning `self`
* `transform()`
* `fit_transform()`

If superclass is `TransformerMixin`, we get `fit_transform()` for free. If superclass is `BaseEstimator` we get `get_params()` and `set_params()` which are useful for automatic hyperparameter tuning.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributeAdder(BaseEstimator, TransformerMixin):
    
    # Note: No *args or **kwargs because of the base
    # estimator subclass (for some reason)
    def __init__(self, add_bedrooms_per_room = True,
                 rooms_i = rooms_ix,
                 bedrooms_i = bedrooms_ix,
                 population_i = population_ix,
                 household_i = household_ix):
        self.add_bedrooms_per_room = add_bedrooms_per_room
        self.rooms_i = rooms_i
        self.bedrooms_i = bedrooms_i
        self.population_i = population_i
        self.household_i = household_i
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, self.rooms_i] / X[:, self.household_i]
        population_per_household = X[:, self.population_i] / X[:, self.household_i]
        
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, self.bedrooms_i] / X[:, self.rooms_i]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributeAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

## Combining Transformers with Pipelines

Scikit-learn provides the class `Pipeline` which simply combines transformers implementing `fit()` and `transform()`. It will apply them in the order that you provide, and apparently, the last transformers only need to implement `fit()`.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("attrib_adder", CombinedAttributeAdder(add_bedrooms_per_room=True)),
    ("std_scaler", StandardScaler())
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

### Pipeline That Takes a Pandas Object

In [ ]:
# Make a class that takes pandas and converts to numpy
class AlphabeticalDataFrameSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        self.attribute_names.sort()        
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].to_numpy()
    
num_columns = list(housing_num.columns)
cat_columns  = ["ocean_proximity"]
categories = sorted(list(housing["ocean_proximity"].unique()))

# Pipeline to process the numerical values
num_pipeline = Pipeline([
    ("selector", AlphabeticalDataFrameSelector(num_columns)),
    ("imputer", SimpleImputer(strategy="median")),
    ("attrib_adder", CombinedAttributeAdder()),
    ("std_scaler", StandardScaler())
])

# Pipeline to process the categorical values
cat_pipeline = Pipeline([
    ("selector", AlphabeticalDataFrameSelector(cat_columns)),
    ("cat_encoder", OneHotEncoder(categories=[categories],dtype=np.int16))
])

# In the OneHotEncoder transform, we must provide the sorted list of expected
# categories (for each feature, hence a nested list) so the pipeline 
# ***always*** generates the expected size for the one-hot-encoding sparse
# matrix.

### Joining/Concatenating Pipeline Results

Scikit-learn provides the `FeatureUnion` class which runs all the individual pipelines/transformer provided in parallel and then concatenates the result of them.

In [ ]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_piepline", cat_pipeline)
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared.toarray().shape

## Serialising Models

As we can see, some of these models take quite a long time to train (RandomDecisionRegressor), 
so saving them for future uses would be good.

In [ ]:

import os.path
from joblib import dump, load

def dump_model(model, model_name):
    
    # Create model folder if necessary
    if not os.path.exists("models") or not os.path.isdir("models"):
        print("Making dir")
        os.mkdir("models")
        
    # dump the file if it already does not exist
    path = "models/{:s}".format(model_name)
    path = os.path.join("models", model_name)
    if not os.path.exists(path) and not os.path.isdir(path):
        dump(model, path)
        
def load_model(model_name):
    
    # Only load if valid
    path = os.path.join("models", model_name)
    if os.path.exists(path) and not os.path.isdir(path):
        return load(path)
    return None

## Training the Model - Linear Regression

Training is easy with scikit-learn. LinearRegression is an estimator, providing a `fit()` function and `predict()` functions.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = load_model("lin_reg.pkl")
if not lin_reg:
    print("Training the model...")
    # Training/fitting the model
    lin_reg = LinearRegression()
    lin_reg.fit(housing_prepared, housing_labels)

# Testing / predicting with the model
some_data = housing.iloc[:10]
some_labels = housing_labels[:5]
some_data_prepared = full_pipeline.fit_transform(some_data)

for pred, actual in zip(lin_reg.predict(some_data_prepared), list(some_labels)):
    print("Predicted {:10.2f};  Actual: {:10.2f}".format(pred, actual))

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

### Training the Model - Decision Tree Regressor

As we can see in mean square error above, each prediction has an average of £69,000 error, which does not make it great as a price predictor. Let's try a decision tree regressor.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = load_model("tree_reg.pkl")
if not tree_reg:
    print("Training model")
    # Fit and predict the labels
    tree_reg = DecisionTreeRegressor()
    tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

## Validating Models with Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
scores_tree = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error",
                              cv=10, n_jobs=4)
tree_rmse_scores = np.sqrt(-scores_tree)

scores_linear = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error",
                                cv=10, n_jobs=4)
linear_rmse_scores = np.sqrt(-scores_linear)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Sandard deviation:", scores.std())
    
print("SCORES FOR DECISION TREE")
display_scores(tree_rmse_scores)
print()
print("SCORES FOR LINEAR REGRESSION")
display_scores(linear_rmse_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = load_model("forest_reg.pkl")
if not forest_reg:
    forest_reg = RandomForestRegressor()
    forest_reg.fit(housing_prepared, housing_labels)

scores_random = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10, n_jobs=4)
random_rmse_scores = np.sqrt(-scores_random)

print("SCORES FOR RANDOM FOREST")
display_scores(random_rmse_scores)

## Fine-Tuning Model Parameters with Grid Search

Scikit-learn's `GridSearchCV` seems like a brute-force estimator which tries all combinations of hypter-parameters you give. Pick the best parameters by analysing the cross-validation performance.

For bigger searches, see `RandomizedSearchCV`

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_estimators": [3, 10, 30], "max_features": [2, 3, 6, 8]},
    {"bootstrap": [False], "n_estimators": [3, 10], "max_features": [2, 3, 4]}
]

forest_grid_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_grid_reg, param_grid, cv=5,
                           scoring="neg_mean_squared_error", n_jobs=4)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# Best estimator is saved in best_estimator_
print("Best estimator params are:", grid_search.best_params_)
best_estimator = grid_search.best_estimator_

# Print the performance of all the cross-validations
cv_results = grid_search.cv_results_
for neg_mean, params in zip (cv_results["mean_test_score"], cv_results["params"]):
    mean = np.sqrt(-neg_mean)
    print("Mean error is: {:10.2f}; Params are {:s}".format(mean, str(params)))

### Analysing Best Fetures

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]

# we can get the transformer from a pipeline
# with this
cat_encoder = cat_pipeline.named_steps["cat_encoder"]
cat_onehot_attribs = list(cat_encoder.categories_[0])

attributes = sorted(num_columns) + extra_attribs + cat_onehot_attribs
importance = best_estimator.feature_importances_
importance_attributes = sorted(zip(attributes, importance), key=lambda x: x[1], reverse=True) 
for attrib, importance in importance_attributes:
    print("Feature Importance of {:18s} is {:7.4f}".format(attrib, importance))

# Exercises

The cells below will implement each of the exerises for chapter 1.

### Exercise 1 - Using a SVM Regressor as the Classifier

In [ ]:
from sklearn.svm import SVR

svr_object = SVR()
svr_grid_params = [
    {"kernel": ["linear"], "C": [0.1, 0.5, 1.0, 3.0], "gamma": ["scale", "auto"]},
    {"kernel": ["rbf"], "C": [0.1, 0.7, 1.5, 3.0, 4.0], "gamma": ["scale", "auto"]},
    {"kernel": ["poly"], "degree": [2, 3, 4, 5], "C": [0.5, 0.7, 3.0, 5.0, 7.0], "gamma": ["auto"]}
]

svr_grid = GridSearchCV(svr_object, svr_grid_params, cv=5,
                           scoring="neg_mean_squared_error", n_jobs=4)
svr_grid.fit(housing_prepared, housing_labels)


svr_cv_results = svr_grid.cv_results_
for neg_mean, params in zip(svr_cv_results["mean_test_score"], svr_cv_results["params"]):
    mean = np.sqrt(-neg_mean)
    print("Mean error is: {:10.2f}; Params are {:s}".format(mean, str(params)))

### Exercise 2 - Using the RandomizedSearchCV 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

svr_object = SVR()
svr_dist = dict(C=uniform(loc=0.1, scale=5))
svr_rand_search = RandomizedSearchCV(svr_object, svr_dist, n_jobs=4, cv=5, scoring="neg_mean_squared_error")
svr_rand_search.fit(housing_prepared, housing_labels)

svr_rand_results = svr_rand_search.cv_results_
for neg_mean, params in zip(svr_rand_results["mean_test_score"], svr_rand_results["params"]):
    mean = np.sqrt(-neg_mean)
    print("Mean error is: {:10.2f}; Params are {:s}".format(mean, str(params)))

### Exercise 3 - Adding Best Feature Transformer to Pipeline

In [ ]:
from sklearn.pipeline import make_pipeline

class BestFeatureSelector(BaseEstimator, TransformerMixin ):
    
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return X[self.features].to_numpy()
    
best_features = ["median_income",
                "bedrooms_per_room",
                "longitude",
                "latitude",
                "pop_per_hhold"]


starting_features = ["total_rooms",
                     "total_bedrooms",
                     "population",
                     "households",
                     "median_income"]
                     
# 1st stage = housing[starting_features]
# 2nd stage = housing[starting_features + ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]]
ex_num_pipeline = make_pipeline(BestFeatureSelector(starting_features),
                                SimpleImputer(strategy="median"),
                                CombinedAttributeAdder(rooms_i=0, bedrooms_i=1, population_i=2, household_i=3),
                                StandardScaler())

ex_cat_pipeline = make_pipeline(BestFeatureSelector(["ocean_proximity"]),
                               OneHotEncoder(categories=[categories],dtype=np.int16))

ex_union_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", ex_num_pipeline),
    ("cat_piepline", ex_cat_pipeline)
])

ex_housing_prepared = ex_union_pipeline.fit_transform(housing, housing_labels)[:, [4, 5, 6, 7, 9]]

In [ ]:

svr_object = SVR()
svr_grid_params = [
    {"kernel": ["linear"], "C": [0.1, 0.5, 1.0, 3.0], "gamma": ["scale", "auto"]},
    {"kernel": ["rbf"], "C": [0.1, 0.7, 1.5, 3.0, 4.0], "gamma": ["scale", "auto"]},
    {"kernel": ["poly"], "degree": [2, 3, 4, 5], "C": [0.5, 0.7, 3.0, 5.0, 7.0], "gamma": ["auto"]}
]

svr_grid = GridSearchCV(svr_object, svr_grid_params, cv=5,
                           scoring="neg_mean_squared_error", n_jobs=4)
svr_grid.fit(ex_housing_prepared, housing_labels)


svr_cv_results = svr_grid.cv_results_
for neg_mean, params in zip(svr_cv_results["mean_test_score"], svr_cv_results["params"]):
    mean = np.sqrt(-neg_mean)
    print("Mean error is: {:10.2f}; Params are {:s}".format(mean, str(params)))